In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1" # NVIDIA GeForce RTX 3090
# os.environ["CUDA_VISIBLE_DEVICES"]="7" # NVIDIA GeForce RTX 2080

import tensorflow.compat.v2 as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices('GPU')

print(gpus)

2023-09-11 19:57:13.282654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 19:57:14.046841: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-11 19:57:16.537597: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/Anaconda3/envs/model_maker_object_detection_tutorial/lib/:/data/Anaconda3/envs/kiwibes/lib/
2023-09-11 19:57:16.537802: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not loa

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import glob, os, shutil, re, random
import matplotlib.pyplot as plt
import numpy as np
import h5py

import random
import enum
import numpy as np
import cv2
import imageio as iio
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
from skimage import data, draw, io
import os, shutil
import glob, re
from scipy.interpolate import RegularGridInterpolator
import pandas as pd
import keras

from functions import *
from parameters import *


In [3]:
print(side)

2048


# Input

In [4]:
batch_size = calculate_batch_size(side)
print('batch_size:', batch_size)

batch_size: 2


# Clean start

In [5]:
print(folderModel, os.path.exists(folderModel))
if os.path.exists(folderModel):
    shutil.rmtree(folderModel)
os.mkdir(folderModel)

/data/home/jorism/models/kiwibes2023-unet-006-2048x2048-11_contractions False


# Generators

In [6]:
# Make generators
train_gen = create_hdf5_generator(fileHDFTrain, batch_size, keys = ['features', 'targets'])
test_gen  = create_hdf5_generator(fileHDFTest,  batch_size, keys = ['features', 'targets'])

In [7]:
# # Check train generator one batch; run this several times to see different batches
# features, targets = next(train_gen)

# for i in range(batch_size):
#     n_img_feat = features.shape[-1]//3
#     plt.figure(figsize=(20,8))
#     for s in range(n_img_feat):
#         subplot_num = 100 + 10*n_img_feat + 10 + s + 1
#         plt.subplot(subplot_num) 
#         plt.imshow(features[i, :, :, s*3:s*3+3])
#     plt.subplot(subplot_num + 1)
#     plt.imshow(targets[i])
#     plt.show() 

In [8]:
# # Check test generator one batch; run this several times to see different batches
# features, targets = next(test_gen)

# for i in range(batch_size):
#     n_img_feat = features.shape[-1]//3
#     plt.figure(figsize=(20,8))
#     for s in range(n_img_feat):
#         subplot_num = 100 + 10*n_img_feat + 10 + s + 1
#         plt.subplot(subplot_num) 
#         plt.imshow(features[i, :, :, s*3:s*3+3])
#     plt.subplot(subplot_num + 1)
#     plt.imshow(targets[i])
#     plt.show() 

# Model

In [9]:
# Find size of train and test sets
db = h5py.File(fileHDFTrain, 'r')
n_train = db['features'].shape[0]
c_feat  = db['features'].shape[-1]
db.close()

db = h5py.File(fileHDFTest, 'r')
n_test = db['features'].shape[0]
db.close()

print('n_train:', n_train)
print('n_test:', n_test)
print('c_feat:', c_feat)


n_train: 1120
n_test: 284
c_feat: 6


In [10]:
# Calculate number of steps per epoch for train and validation
steps_per_epoch = n_train//batch_size
validation_steps = n_test//batch_size

print('epochs:', epochs)
print('steps_per_epoch:', steps_per_epoch)
print('validation_steps:', validation_steps)

epochs: 25
steps_per_epoch: 560
validation_steps: 142


In [11]:
# Make model

data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomBrightness(factor=(-0.2, 0.2))
    ])

callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
            # tf.keras.callbacks.TensorBoard(log_dir='logs')
    ]


for m in range(10):
    print()
    print('TRAINING MODEL', m, folderModel)

    # model = initiate_model_old(data_augmentation = data_augmentation)
    model = initiate_model(
        h_img = h_img, 
        w_img = w_img, 
        c_feat = c_feat,
        num_classes = num_classes,
        n_contractions = n_contractions,
        data_augmentation = data_augmentation
    )

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


    model.fit(
        train_gen,
        validation_data=(test_gen),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        steps_per_epoch = steps_per_epoch,
        validation_steps = validation_steps
    )

    fileModel = f'{folderModel}/model_{str(m).zfill(3)}.hdf5'
    print(fileModel)
    model.save(fileModel)

    fileLosses = f'{folderModel}/losses_{str(m).zfill(3)}.csv'
    print(fileLosses)
    pd.DataFrame({
        'train_loss' : model.history.history['loss'],
        'val_loss' : model.history.history['val_loss']
    }).to_csv(fileLosses)

2023-09-11 19:57:21.619046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 19:57:23.687033: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22297 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1e:00.0, compute capability: 8.6



TRAINING MODEL 0 /data/home/jorism/models/kiwibes2023-unet-006-2048x2048-11_contractions


2023-09-11 19:57:36.970992: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.00GiB (rounded to 9663676416)requested by op StatelessTruncatedNormalV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-09-11 19:57:36.971077: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-09-11 19:57:36.971110: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 39, Chunks in use: 39. 9.8KiB allocated for chunks. 9.8KiB in use in bin. 2.8KiB client-requested in use in bin.
2023-09-11 19:57:36.971137: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 6, Chunks in use: 6. 3.0KiB allocated for chunks. 3.0KiB in use in bin. 3.0KiB client-requested in use in bin.
2023-09-11 19:57:3

ResourceExhaustedError: {{function_node __wrapped__StatelessTruncatedNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[3,3,16384,16384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessTruncatedNormalV2]

In [ ]:
# train_loss = model.history.history['loss']
# val_loss   = model.history.history['val_loss']
        
# plt.figure()
# plt.plot(train_loss, 'r',  label='Train')
# plt.plot(val_loss,   'bo', label='Validation')
# plt.title('Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss Value')
# plt.ylim(bottom=0)
# plt.legend()
# plt.show()



In [ ]:
# train_loss_no_aug = model.history.history['loss']
# val_loss_no_aug   = model.history.history['val_loss']

# plt.figure(figsize=(20,6))

# plt.subplot(121)
# plt.plot(train_loss, 'r',  label='Train')
# plt.plot(val_loss,   'bo', label='Validation')
# plt.title('Augmenation')
# plt.xlabel('Epoch')
# plt.ylabel('Loss Value')
# plt.ylim(bottom=0)
# plt.legend()

# plt.subplot(122)
# plt.plot(train_loss_no_aug, 'r',  label='Train')
# plt.plot(val_loss_no_aug,   'bo', label='Validation')
# plt.title('No augmentation')
# plt.xlabel('Epoch')
# plt.ylabel('Loss Value')
# plt.ylim(bottom=0)
# plt.legend()

# plt.show()

# Load and plot losses

In [ ]:
print(folderModel)
! ls {folderModel}

In [ ]:
# plt.figure(figsize=(20,10))

# for file in glob.glob(f'{folderModel}/losses_*'):
#     data = pd.read_csv(file)
    
#     plt.subplot(121)
#     plt.plot(data['train_loss'])
#     plt.ylim([0,0.2])
    
#     plt.subplot(122)
#     plt.plot(data['val_loss'])
#     plt.ylim([0,0.2])
    
# plt.show()

# Check: load model from file and predict something

In [ ]:
# new_model = tf.keras.models.load_model(fileModel)

In [ ]:
# folderData = '/data/home/jorism/data/kiwibes2023-camera_158-2023-06-05T12-tagged/'

# filesTxt = glob.glob(f'{folderData}/*.txt')

# f = 0
# fileJpg = filesTxt[f][:-3] + 'jpg'
# image_full = plt.imread(fileJpg)/255

# plt.figure()
# plt.imshow(image_full)
# plt.show()

In [ ]:
# # int(np.ceil(image_full.shape[0]/h_img))

# for xmin in np.arange(0, image_full.shape[0], h_img):
#     for ymin in np.arange(0, image_full.shape[1], w_img):
#         print(xmin,ymin)
#         image = np.zeros((w_img, h_img, 3))
#         crop = image_full[xmin:xmin+w_img, ymin:ymin+h_img, :]
#         image[0:crop.shape[0], 0:crop.shape[1], :] = crop
        
#         prediction = new_model.predict(image[tf.newaxis, ...])[0]
#         predicted_mask = (prediction > 0.5).astype(np.uint8)

#         plt.figure(figsize=(20,20))
#         plt.subplot(121)
#         plt.imshow(image)
#         plt.subplot(122)
#         plt.imshow(predicted_mask)
#         plt.show()
        
#         # plt.figure()
#         # plt.imshow(image)
#         # plt.show()

